In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


In [8]:
q = '''
SELECT id,
       UID,
       name,
       keyword,
       category_id,
       status
FROM service1_quicket.product_info
WHERE create_date >= dateadd(WEEK, -4, getdate())
'''

df = pd.read_sql(q, con=bun_dw)

In [6]:
df.shape

(1147947, 5)

In [22]:
df.groupby('category_1depth').size()

category_1depth
100      1354
200      2818
210      2702
220      1562
230       878
240      5177
300         1
310    376331
320    331599
400    439955
410     80173
500     65581
600    456176
700    113719
750     64690
800    117340
900    253027
910    179027
999     78961
dtype: int64

In [7]:
df.head()

,id,uid,name,keyword,category_id
0,149500416,1376343,폴로 와플 라운드넥티/t108,폴로와플티,320010100
1,149502464,96807,솔리드옴므 테이핑 맥자켓 50사이즈,"솔옴,우영마,트렌치코트,버버리,50사이즈",320100200
2,149505024,75479827,국내)구찌 자수 멀티카라티 수입/6만,"명품가방,명품지갑,명품신발,명품의류,명품여성가방",310120030
3,149506048,1639399,103-105 올세인츠 흑청 커팅 데님셔츠,"타임,산드로,이타우츠,시스템옴므,한섬",320040400
4,149507584,7046199,원피스,"프리사이즈,원피스",310120030


In [20]:
df['category_1depth'] = df['category_id'].str[:3]

In [46]:
import re
# import string

def preprocess(name):
    name = str(name)
    name = re.sub(r'\s+', ' ', name)
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       "]+", flags = re.UNICODE)
#     name = name.translate(string.maketrans("",""), string.punctuation)
    name = regrex_pattern.sub(r'',name)
    name = re.sub(r'[^\w\s]', ' ', name)
    return name.lower().strip()

In [47]:
df['name_preprocessed'] = df.name.map(preprocess)

In [49]:
df[['name', 'name_preprocessed']].head(50)

,name,name_preprocessed
0,폴로 와플 라운드넥티/t108,폴로 와플 라운드넥티 t108
1,솔리드옴므 테이핑 맥자켓 50사이즈,솔리드옴므 테이핑 맥자켓 50사이즈
2,세븐틴 디노 포카 팔아용,세븐틴 디노 포카 팔아용
3,네파바람막이,네파바람막이
4,삼> 아이폰 중고폰 파손폰 연체폰 <삼,삼 아이폰 중고폰 파손폰 연체폰 삼
5,DJI 매빅 2 줌 + 플라이모어 키트 팝니다,dji 매빅 2 줌 플라이모어 키트 팝니다
6,국내)구찌 자수 멀티카라티 수입/6만,국내 구찌 자수 멀티카라티 수입 6만
7,103-105 올세인츠 흑청 커팅 데님셔츠,103 105 올세인츠 흑청 커팅 데님셔츠
8,에어팟 1 본체 팝니다,에어팟 1 본체 팝니다
9,원피스,원피스


In [50]:
df.to_csv('df.csv', index=False)

In [51]:
df[df['category_1depth'].isin(['310', '320', '400'])]

,id,uid,name,keyword,category_id,status,category_1depth,name_preprocessed
0,149500416,1376343,폴로 와플 라운드넥티/t108,폴로와플티,320010100,0,320,폴로 와플 라운드넥티 t108
1,149502464,96807,솔리드옴므 테이핑 맥자켓 50사이즈,"솔옴,우영마,트렌치코트,버버리,50사이즈",320100200,0,320,솔리드옴므 테이핑 맥자켓 50사이즈
6,149505024,75479827,국내)구찌 자수 멀티카라티 수입/6만,"명품가방,명품지갑,명품신발,명품의류,명품여성가방",310120030,2,310,국내 구찌 자수 멀티카라티 수입 6만
7,149506048,1639399,103-105 올세인츠 흑청 커팅 데님셔츠,"타임,산드로,이타우츠,시스템옴므,한섬",320040400,0,320,103 105 올세인츠 흑청 커팅 데님셔츠
9,149507584,7046199,원피스,"프리사이즈,원피스",310120030,0,310,원피스
...,...,...,...,...,...,...,...,...
2571062,150935065,2650528,나이키 트레이닝복 퓨추라 클럽 팬츠 AR9846-063 삽니다,"나이키,그레이,m,s",310190020,0,310,나이키 트레이닝복 퓨추라 클럽 팬츠 ar9846 063 삽니다
2571066,150935076,12431375,<신상> 디올 라탄 크로스백,"디올,크리스챤디올,디올신상,디올오블리크,디올라탄백",400010200,0,400,신상 디올 라탄 크로스백
2571067,150935080,75110491,드라이프 DRIFE pvc mini crossbag 미니크로스백 프리즘,"프리즘백,크로스백,드라이프가방,드라이프크로스백,홀로그램백",400010200,0,400,드라이프 drife pvc mini crossbag 미니크로스백 프리즘
2571068,150935081,12632471,아미 반팔 세트,"아미,반팔,세트,반바지,명품",320,0,320,아미 반팔 세트


In [60]:
brand = pd.read_csv('../brand_dictionary/fashion_brand.csv')


In [65]:
brand.iloc[:30, :2]

,brand,brand_in_english
0,스톤아일랜드,Stone island
1,나이키,nike
2,프라이탁,freitag
3,톰브라운,THOM BROWNE
4,스투시,stussy
5,스타벅스,starbucks
6,루이비통,louis vuitton
7,레고,lego
8,아크테릭스,"arc'teryx, ARC TERYX"
9,폴로,POLO


In [80]:
eng = brand.iloc[:30, :2].brand_in_english.str.lower()

In [81]:
for i in eng:
    print(str(i).split(','))

['stone island']
['nike']
['freitag']
['thom browne']
['stussy']
['starbucks']
['louis vuitton']
['lego']
["arc'teryx", ' arc teryx']
['polo']
['gucci']
['lucky chouette']
['ader error']
['andersson bell']
['burberry']
['our legacy']
['chanel']
['off white']
['nan']
['balenciaga']
['wooyoungmi']
['moncler']
['golden goose']
['maison kitsune', ' maison kitsuné']
['prada']
['supreme']
['zara']
['dior']
['vespa']
['galaxy']


In [87]:
brand = brand.iloc[:30, :2]

,brand,brand_in_english
0,스톤아일랜드,Stone island
1,나이키,nike
2,프라이탁,freitag
3,톰브라운,THOM BROWNE
4,스투시,stussy
5,스타벅스,starbucks
6,루이비통,louis vuitton
7,레고,lego
8,아크테릭스,"arc'teryx, ARC TERYX"
9,폴로,POLO


In [91]:
names = df.name_preprocessed.tolist()

In [98]:
brand_dict = {b: [] for b in brand.brand}

In [100]:
for name in names:
    for b in brand.brand:
        if b in name: 
            values = brand_dict.get(b, [])
            values.append(name)
            brand_dict[b] = values

In [105]:
total_n = 0
for b in brand_dict:
    n = len(brand_dict[b])
    total_n += n
    print(b + ' ' + str(n))
print('=====')
print(total_n)

스톤아일랜드 13309
나이키 81410
프라이탁 2110
톰브라운 10295
스투시 5421
스타벅스 15194
루이비통 25241
레고 4078
아크테릭스 1021
폴로 26153
구찌 29597
럭키슈에뜨 1253
아더에러 766
앤더슨벨 1663
버버리 13200
아워레가시 575
샤넬 18040
오프화이트 5621
마이멜로디 1759
발렌시아가 11352
우영미 1028
몽클레어 5120
골든구스 2139
메종키츠네 2262
프라다 10461
슈프림 3347
자라 9334
디올 12238
베스파 433
갤럭시 76834
=====
391254


In [106]:
total_n/df.shape[0]

0.15217549418121865

In [157]:
df.shape

(2571071, 9)

In [115]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/emilyPark/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [130]:
s = pd.Series(nltk.ngrams(words, 2)).value_counts()

In [125]:
s.shape

(764835,)

In [135]:
df.head()

,id,uid,name,keyword,category_id,status,category_1depth,name_preprocessed
0,149500416,1376343,폴로 와플 라운드넥티/t108,폴로와플티,320010100,0,320,폴로 와플 라운드넥티 t108
1,149502464,96807,솔리드옴므 테이핑 맥자켓 50사이즈,"솔옴,우영마,트렌치코트,버버리,50사이즈",320100200,0,320,솔리드옴므 테이핑 맥자켓 50사이즈
2,149502720,12878294,세븐틴 디노 포카 팔아용,None,910100007,2,910,세븐틴 디노 포카 팔아용
3,149502976,12022831,네파바람막이,네파키즈,500113006,0,500,네파바람막이
4,149503232,75580667,삼> 아이폰 중고폰 파손폰 연체폰 <삼,"중고폰,아이폰,경산,대구,블랙",600700001,2,600,삼 아이폰 중고폰 파손폰 연체폰 삼


In [136]:
corpus = []
df['word_count'] = df['name'].apply(lambda x: len(str(x).split(" ")))

In [140]:
w_cnt = len(df.word_count)

In [ ]:
import re
# import string

def preprocess(name):
    name = str(name)
    name = re.sub(r'\s+', ' ', name)
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       "]+", flags = re.UNICODE)
#     name = name.translate(string.maketrans("",""), string.punctuation)
    name = regrex_pattern.sub(r'',name)
    name = re.sub(r'[^\w\s]', ' ', name)
    return name.lower().strip()

In [142]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

corpus = []

for i in range(len(df)):
    text = preprocess(df['name'][i])
    text = text.split()
    ps = PorterStemmer()
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text]
    text = " ".join(text)
    corpus.append(text)

In [143]:
corpus

['폴로 와플 라운드넥티 t108',
 '솔리드옴므 테이핑 맥자켓 50사이즈',
 '세븐틴 디노 포카 팔아용',
 '네파바람막이',
 '삼 아이폰 중고폰 파손폰 연체폰 삼',
 'dji 매빅 2 줌 플라이모어 키트 팝니다',
 '국내 구찌 자수 멀티카라티 수입 6만',
 '103 105 올세인츠 흑청 커팅 데님셔츠',
 '에어팟 1 본체 팝니다',
 '원피스',
 '드림캐쳐 오드아이 특전 유현 포카 포토카드',
 '디즈니 리멘트 꿈과 마법의 레스토랑 단품 7번',
 '뉴발란스 327 문빔 250 새제품',
 '트레이닝하의',
 '테디베어 작은크기 택포',
 '아이폰x 256기가 실버 정상해지 공기계 30만 1 제품정보 확',
 '샤넬j12 38mm 티타늄세라믹 그레이',
 '타미진스 긴팔티',
 '현대 제네시스 중고차팝니다 은색 문의 01053491942',
 '폴로 랄프로렌 프리미엄 라운드 가디건',
 '20년식 메리다 빅나인500',
 '스와로브스키목걸이',
 '새상품 디올 하이탑 스니커즈',
 '하이큐 쿠로오 테츠로 족자봉 슬림 태피스트리',
 '방탄소년단 친필 싸인 앨범 양도',
 '코치 가방',
 '특가세일 셀린느가방 카바스백미니',
 '사회심리학 교재',
 '에어팟프로 개봉품 팔아요',
 '새상품 에르메스 클릭아슈 팔찌',
 '더페이스샵 보석반지 틴트 3호 블링 핑크',
 '남자 골프바지 80사이즈 32인치 새제품',
 '앤던슨벨 아식스 콜라보 255',
 '나이키 에어맥스 97',
 '이누야샤 극장판 팝업노트',
 '필름카메라 미놀타 리바줌70 판매합니다',
 '삼 아이폰 중고폰 파손폰 연체폰 삼',
 '너무 쉬운 매경테스트',
 '메이빈스 코튼밴딩pt',
 '구함 산리오 카페시리즈 피아노인형 구합니다',
 '콘스탄틴 버나드 2016',
 '새상품 맥북 13',
 '세트가격 나이키 조던 파리스 후드집업 트레이닝세트 맨투맨 바지 반팔티',
 '파라점퍼스 라이트핸드',
 '산본점 바디채널 pt 6회 양도 해요',
 '아이폰 박스폰 가개통 파손폰 중고폰

In [161]:
df[40:60]

,id,uid,name,keyword,category_id,status,category_1depth,name_preprocessed,word_count
40,149554176,75658997,콘스탄틴 버나드 2016,"510,콘스탄틴,버나드,어베인,엔진11",700350100,1,700,콘스탄틴 버나드 2016,3
41,149555968,75832777,새상품)맥북 13,None,600100001,2,600,새상품 맥북 13,2
42,149556736,75074592,세트가격 나이키 조던 파리스 후드집업 트레이닝세트 맨투맨 바지 반팔티,"맨투맨,바지,트레이닝세트,반팔티반바지",320020100,2,320,세트가격 나이키 조던 파리스 후드집업 트레이닝세트 맨투맨 바지 반팔티,9
43,149556992,5118599,파라점퍼스 라이트핸드,"m,미디움,파라점퍼스,패딩,명품",320080200,0,320,파라점퍼스 라이트핸드,2
44,149557760,8353109,산본점 바디채널 pt 6회 양도 해요,"바디채널양도,바디채널,바디채널산본,헬스",700650999,0,700,산본점 바디채널 pt 6회 양도 해요,6
45,149558272,74595790,아이폰/박스폰/가개통/파손폰/중고폰매입/출장매입,None,600700001,0,600,아이폰 박스폰 가개통 파손폰 중고폰매입 출장매입,1
46,149558528,3529126,[정품] 살바토레 페레가모 트레이시 숄더백 21-E298,"루이비통,샤넬,구찌,프라다,토트백",400010100,3,400,정품 살바토레 페레가모 트레이시 숄더백 21 e298,6
47,149558784,9039101,(새상품) 루이x 월릿 반지갑 최저가 택포,"루이비통,반지갑,레플,루이비통지갑,남자지갑",400060400,2,400,새상품 루이x 월릿 반지갑 최저가 택포,6
48,149559808,74257379,뉴이스트W 트와이스 세븐틴 갓세븐 앨범,"뉴이스트W앨범,트와이스앨범,세븐틴앨범,갓세븐앨범",910100004,0,910,뉴이스트w 트와이스 세븐틴 갓세븐 앨범,5
49,149561344,5974400,에어팟 젤리 케이스 3개,"에어팟케이스판매,에어팟젤리케이스,에어팟젤리,에어팟케이스,케이스",600700003,0,600,에어팟 젤리 케이스 3개,4


In [153]:
from nltk.util import ngrams

ng = ngrams(' '.join(corpus).split(), 2)

TypeError: 'generator' object is not subscriptable